In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix, classification_report
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
image_columname = 'filename'

In [3]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

# replacing spaces with underscores:
available_classes = [cname.replace(' ', '_') for cname in available_classes]

In [4]:
# having it reverse for plot:
available_classes_r = available_classes[::-1]

available_classes_r_upper = [cname.upper() for cname in available_classes_r]

In [5]:
available_models

['ViT-bigG-14-CLIPA-336',
 'RN101',
 'RN50x64',
 'ViT-L_14',
 'ViT-B_32',
 'EVA02-E-14-plus',
 'ViT-SO400M-14-SigLIP-384',
 'ViT-H-14-378-quickgelu']

In [6]:
available_classes

['asphalt',
 'cobblestone',
 'compacted',
 'concrete',
 'concrete_plates',
 'grass',
 'gravel',
 'ground',
 'paving_stones',
 'sett']

In [7]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name]).copy()

In [8]:
{model_name: len(modelwise_dfs[model_name]) for model_name in modelwise_dfs}

{'EVA02-E-14-plus': 5000,
 'RN101': 5000,
 'RN50x64': 5000,
 'ViT-bigG-14-CLIPA-336': 5000,
 'ViT-B_32': 5000,
 'ViT-H-14-378-quickgelu': 5000,
 'ViT-L_14': 5000,
 'ViT-SO400M-14-SigLIP-384': 5000}

In [9]:
merged_df = pd.concat(all_dfs)

In [10]:
true_classes = merged_df.groupby(image_columname).agg('first')[class_label] #.str.replace('_', ' ')
merged_pred = merged_df.groupby(image_columname).sum(numeric_only=True)

merged_pred[prediction_label] = merged_pred.idxmax(axis=1)
merged_pred[class_label] = true_classes


In [11]:
merged_pred

,asphalt,cobblestone,compacted,concrete,concrete_plates,grass,gravel,ground,paving_stones,sett,prediction,class
filename,,,,,,,,,,,,
000111e2b580f09ee3c82e741e5fc402.png,1.119566,0.626668,0.315340,1.067032,0.571635,0.535103,1.142461,2.085953,0.282286,0.254142,ground,compacted
00098db7c6cd6ee36d590ff0d5db7ad1.png,0.173628,0.301090,0.151611,4.517852,0.835243,0.017109,0.375464,0.582497,1.040450,0.004802,concrete,concrete
000a02ec8efd2faef1315a54af697dcc.png,2.389232,0.799234,0.210236,0.417047,0.689314,0.430098,0.552883,1.448259,0.545268,0.518487,asphalt,concrete_plates
001449cde8a4959f753aa0e5e4fe1166.png,0.105323,4.306164,0.028392,0.075611,0.035851,0.009307,2.192703,0.402684,0.827460,0.016294,cobblestone,cobblestone
00157343508e08f1dbc0efc1f8066f3d.png,0.183626,0.938241,1.219646,0.116098,0.209043,0.352456,1.834101,1.943612,1.109219,0.093870,ground,ground
...,...,...,...,...,...,...,...,...,...,...,...,...
ffd4bc27fe750ca5fedac61da4f7a397.png,0.214579,0.517436,1.046109,0.079391,0.429189,0.172434,1.197481,3.117920,1.065587,0.160118,ground,ground
ffdf02c1935b18874b5b6d30b163f5aa.png,0.437350,0.380334,0.249078,2.987426,1.886027,0.086050,0.648325,0.557583,0.703132,0.064330,concrete,compacted
ffeebe48ab30925a6b36cd74dd6228c4.png,0.575448,0.543808,0.473006,2.530337,0.915317,0.603612,0.980028,0.552140,0.178257,0.648179,concrete,concrete


In [12]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,
                         predicted_labels,
                         labels=available_classes_r,
                           normalize='true'
                           ) * 100
c_mat = c_mat.round(3)

In [13]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [14]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes_r,
    output_dict=True
)

{'sett': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 500.0},
 'paving_stones': {'precision': 0.5442739079102715,
  'recall': 0.922,
  'f1-score': 0.6844840386043058,
  'support': 500.0},
 'ground': {'precision': 0.4384920634920635,
  'recall': 0.884,
  'f1-score': 0.5862068965517241,
  'support': 500.0},
 'gravel': {'precision': 0.7783687943262412,
  'recall': 0.878,
  'f1-score': 0.825187969924812,
  'support': 500.0},
 'grass': {'precision': 0.939047619047619,
  'recall': 0.986,
  'f1-score': 0.9619512195121951,
  'support': 500.0},
 'concrete_plates': {'precision': 0.6164383561643836,
  'recall': 0.09,
  'f1-score': 0.15706806282722513,
  'support': 500.0},
 'concrete': {'precision': 0.5218023255813954,
  'recall': 0.718,
  'f1-score': 0.6043771043771043,
  'support': 500.0},
 'compacted': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 500.0},
 'cobblestone': {'precision': 0.5090470446320868,
  'recall': 0.844,
  'f1-score': 0.6350639578630549,

In [15]:
n_rows = int(ceil(len(available_models)/2))

In [16]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    true_labels =      modelwise_dfs[model][class_label]#.tolist()
    predicted_labels = modelwise_dfs[model][prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(
        true_labels,
        predicted_labels,
        labels=available_classes,
        normalize='true'
        ) *  100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

In [17]:
modelwise_confusion_matrices

{'EVA02-E-14-plus': array([[58.6,  4.2,  0. , 23.4,  0.8,  3.8,  2.6,  4.8,  0. ,  1.8],
        [ 0.6, 72.2,  0. ,  0. ,  0.2,  0. , 15.4,  1.6, 10. ,  0. ],
        [ 6. ,  0.4,  7. , 48.6,  0.4,  0.6,  7. , 29.6,  0.2,  0.2],
        [13. ,  0.2,  0.4, 65.8,  0.8,  6.4,  6.8,  6.2,  0. ,  0.4],
        [16. , 15.8,  0. , 28.2,  2.6,  4.8,  0. , 19.2, 13.4,  0. ],
        [ 0.4,  0. ,  0. ,  0.4,  0. , 98.4,  0. ,  0.8,  0. ,  0. ],
        [ 4.6,  0. ,  0.2,  0. ,  0. ,  0. , 91.6,  3.4,  0. ,  0.2],
        [ 2.6,  1.4, 27.6,  0.2,  0. ,  0.4, 19.6, 45.6,  0.2,  2.4],
        [ 5.8,  9.8,  0. ,  0.6,  0. ,  0.4,  0.4,  2.8, 80.2,  0. ],
        [ 1.6, 61.6,  0. ,  0.8,  0. ,  0.6,  1.4,  4.6, 29.4,  0. ]]),
 'RN101': array([[26. ,  0. , 11.6,  0. , 29.8,  1.2, 12.4, 10.4,  4.6,  4. ],
        [ 0.6, 62. ,  1.4,  0. ,  0.4,  0.4, 17.4,  0.6, 17.2,  0. ],
        [ 6.6,  3.2, 14. ,  0.2, 24.2,  0. , 14.2, 21.4, 15.8,  0.4],
        [ 2. ,  0.6, 21. ,  0.6, 47.2,  2. ,  5.8,  3.6, 17.

In [18]:
# fig = go.Figure().set_subplots(rows=n_rows, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.05, horizontal_spacing=0.01)
fig = make_subplots(rows=n_rows, cols=2, 
                    shared_xaxes=True, 
                    shared_yaxes=True,
                    subplot_titles=list(modelwise_confusion_matrices.keys()),
                    vertical_spacing=0.015, 
                    horizontal_spacing=0.01,
                    # x_title="Prediction",
                    # y_title="Class",
                    )

for i,modelname in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[modelname],
                
                x=available_classes,
                y=available_classes,
              text_auto=True, 
              #  aspect="auto",
               )
    
    # print(i//2+1,i%2+1)
    subplot['layout']['xaxis']['autorange'] = "reversed"

    fig.add_trace(subplot.data[0], row=i//2+1, col=i%2+1)
    fig.update_xaxes(row=i//2+1, col=i%2+1, autorange='reversed')
    # fig.append_trace(subplot.data[0], row=i//2+1, col=i%2+1)

fig.update_coloraxes(colorscale=px.colors.sequential.Burg)
fig.update_layout(height=400*n_rows, width=400*2,
                  xaxis_title="Prediction",
                  yaxis_title="Class",
                  showlegend=True,)
fig.show()


In [19]:
# getting the diagonal sum of the confusion matrix:
diagonals = {}

for model in modelwise_confusion_matrices:
    diagonals[model] = modelwise_confusion_matrices[model].diagonal().sum()

diagonals['combined'] = c_mat.diagonal().sum() 

diagonals_df = pd.DataFrame.from_dict(diagonals, orient='index',columns=['score']).reset_index().rename(columns={'index':'model_name'}).sort_values(by='score', ascending=False)

diagonals_df['score'] = round(100 *  diagonals_df['score'] / (len(available_models) * 100),2)

diagonals_df

,model_name,score
8,combined,74.45
3,ViT-bigG-14-CLIPA-336,70.05
5,ViT-H-14-378-quickgelu,67.32
0,EVA02-E-14-plus,65.25
7,ViT-SO400M-14-SigLIP-384,60.80
6,ViT-L_14,52.23
1,RN101,46.82
2,RN50x64,41.12
4,ViT-B_32,32.10


In [20]:
fig = px.bar(diagonals_df,x='model_name', y='score', text_auto=True)
fig.show()


In [21]:
selected_models = {}

for modelname in ['ViT-H-14-378-quickgelu', 'ViT-SO400M-14-SigLIP-384', 'EVA02-E-14-plus']:
    selected_models[modelname] = modelwise_dfs[modelname]

In [22]:
merged_selected = pd.concat(selected_models).reset_index()

In [23]:
true_classes = merged_selected.groupby(image_columname).agg('first')[class_label] #.str.replace('_', ' ') #.reset_index().rename(columns={class_label:'class'})

In [24]:
merged_selected = merged_selected.groupby(image_columname).sum(numeric_only=True)

In [25]:
merged_selected.drop(columns=['level_1'], inplace=True)

In [26]:
merged_pred[prediction_label] = merged_selected.idxmax(axis=1)
merged_pred[class_label] = true_classes.tolist()

In [27]:
# computing the confusion matrix:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r,
                          normalize='true'
                          ) * 100
c_mat = c_mat.round(3)

In [28]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [29]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes_r,
    output_dict=True
)

{'sett': {'precision': 0.06451612903225806,
  'recall': 0.004,
  'f1-score': 0.007532956685499058,
  'support': 500.0},
 'paving_stones': {'precision': 0.6398763523956723,
  'recall': 0.828,
  'f1-score': 0.7218831734960767,
  'support': 500.0},
 'ground': {'precision': 0.41395793499043976,
  'recall': 0.866,
  'f1-score': 0.5601552393272963,
  'support': 500.0},
 'gravel': {'precision': 0.9068736141906873,
  'recall': 0.818,
  'f1-score': 0.8601472134595163,
  'support': 500.0},
 'grass': {'precision': 0.9393939393939394,
  'recall': 0.992,
  'f1-score': 0.9649805447470817,
  'support': 500.0},
 'concrete_plates': {'precision': 0.6904761904761905,
  'recall': 0.232,
  'f1-score': 0.3473053892215569,
  'support': 500.0},
 'concrete': {'precision': 0.49930651872399445,
  'recall': 0.72,
  'f1-score': 0.5896805896805897,
  'support': 500.0},
 'compacted': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 500.0},
 'cobblestone': {'precision': 0.4937142857142857,
  'recal

In [30]:
c_mat.diagonal().sum() / (len(available_models))

74.05